In [ ]:
# # ユーザー認証
# from google.colab import auth
# auth.authenticate_user()

#Google setting
from google import colab
colab.drive.mount('/content/gdrive')

#Directory setting
b_dir='./gdrive/My Drive/AI_Seminar/NLP_1/' # Setting working directory
dataset_dir=b_dir+'dataset/'

Mounted at /content/gdrive


In [ ]:
%%bash

free -h

# ツールのダウンロード
apt-get update
apt-get install mecab file swig libmecab-dev mecab-ipadic-utf8
pip install mecab-python3==0.996.5
pip install transformers==2.11.0
pip install tqdm
pip install category_encoders

              total        used        free      shared  buff/cache   available
Mem:            12G        585M        9.7G        1.0M        2.4G         11G
Swap:            0B          0B          0B
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Re

In [ ]:
# insall
import numpy as np
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from scipy import stats
from sklearn.model_selection import KFold
import os,urllib,glob,tarfile
from transformers import BertJapaneseTokenizer, BertModel, BertConfig, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
import category_encoders as ce

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# データの読み込み
train_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S")  # train.csv
valid_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1Mdyv3DPcb1AncxE7H739QWyPkMqcdjqI")  # dev.csv
test_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1yHKfDrs6ZAx4fizToTF7GL7ROvGk7Ry5")  # test.csv
# train_df = pd.read_csv("./dataset/train.csv")
# val_df = pd.read_csv("./dataset/dev.csv")
# test_submission = pd.read_csv("./dataset/test.csv")

train_df.head(5)

,movieName,title,text,rating
0,HERO,楽しみにしてるよ～～!!!,この前、HEROを１話から見直しました!!もぅドキドキで楽しいっ!!あたしも、西遊記見たとき...,5
1,インセプション,期待しすぎた…,DVDでの鑑賞。\nちょっと観て時間がたったので、ざっくりレビューです。\n\n根本的なアイ...,3
2,パッチギ！ LOVE&PEACE,真面目に生きている在日の人たちが可哀想,こんな変な映画が、全国上映されるからまた、在日批判が出てくるんじゃないの？,1
3,ダ・ヴィンチ・コード,いまいちの映画でした,はっきり言って期待しすぎたのが悪かったのか全く面白くなかったです。この手の映画は宗教がらみな...,2
4,踊る大捜査線 THE MOVIE 3 ヤツらを解放せよ！,残念なチャリティーマッチ,制作側の意図や事情を知らずにむやみに酷評するのははばかられますが、はっきり言ってつまらなかっ...,2


In [ ]:
train_df = train_df.dropna()
valid_df = valid_df.dropna()
test_df = test_df.dropna()

In [ ]:
list_cols = ['movieName']
# ce_ohe = ce.OneHotEncoder(cols=list_cols,handle_unknown='impute')
ce_oe = ce.OrdinalEncoder(cols=list_cols, handle_unknown='impute')
# train_df, valid_df, test_df = ce_ohe.fit_transform([train_df, valid_df, test_df])
train_df = ce_oe.fit_transform(train_df)
valid_df = ce_oe.fit_transform(valid_df)
test_df = ce_oe.fit_transform(test_df)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
train_df['text'] = train_df['title'].str.cat(train_df['text'])
valid_df['text'] = valid_df['title'].str.cat(valid_df['text'])
test_df['text'] = test_df['title'].str.cat(test_df['text'])

In [ ]:
train_df['movieName'] = train_df['movieName'].astype(str)
valid_df['movieName'] = train_df['movieName'].astype(str)
test_df['movieName'] = train_df['movieName'].astype(str)

In [ ]:
train_df['text'] = train_df['movieName'].str.cat(train_df['text'])
valid_df['text'] = valid_df['movieName'].str.cat(valid_df['text'])
test_df['text'] = test_df['movieName'].str.cat(test_df['text'])

In [ ]:
# tokenizerの定義とvocabを作成
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
pad = tokenizer.vocab["[PAD]"]  # Padding until Length=512
# max_lengths=256
max_lengths=512

In [ ]:
# 上記のinput_dataの作成方法をもとにdataset化
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels=[]):
        self.input_ids, self.segment_ids, self.attention_masks = [],[],[]
        for text in tqdm(texts):
            token = ["[CLS]"]+tokenizer.tokenize(text)[:max_lengths-2]+["[SEP]"]
            input_id = tokenizer.convert_tokens_to_ids(token)
            segment_id = [0]*max_lengths
            attention_mask = [1]*len(input_id)+[0]*(max_lengths - len(input_id))
            input_id = input_id+[pad]*(max_lengths-len(input_id))
            self.input_ids.append(input_id)
            self.segment_ids.append(segment_id)
            self.attention_masks.append(attention_mask)
        self.input_ids = np.array(self.input_ids)
        self.segment_ids = np.array(self.segment_ids)
        self.attention_masks = np.array(self.attention_masks)
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if len(self.labels):
            return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx], self.labels[idx]
        else:
            return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx]

In [ ]:
# datasetとdataloaderの作成
batch_size = 8  # 8
# X, y = train_df.drop("rating").values, train_df["rating"].values-1
X, y = train_df["text"].values, train_df["rating"].values-1
# X, y = train_df["title"].values, train_df[["rating_1", "rating_2", "rating_3", "rating_4", "rating_5"]].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=100,stratify=y)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=100,stratify=y)
train_ds = ReviewDataset(texts=X_train, labels=y_train)
train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size=batch_size, shuffle=True)
X_validOrg, y_validOrg = valid_df["text"].values, valid_df["rating"].values-1
X_valid, y_valid = np.concatenate([X_val, X_validOrg]), np.concatenate([y_val, y_validOrg])
# X_valid, _, y_valid, _ = train_test_split(X_valid, y_valid, test_size=0.64, random_state=100, stratify=y_valid)
valid_ds = ReviewDataset(texts=X_valid, labels=y_valid)
valid_dl = torch.utils.data.DataLoader(
    valid_ds, batch_size=batch_size, shuffle=False)

In [ ]:
print(X_valid)
print(y_valid)

['10後味最悪後味最悪の映画。\nラストで興ざめしました。\nどうしてこんなラストにしたのか意味不明。\nラストまでは、結構面白かったのに・・・\nラストで全て台無しにしています。'
 '113Dを楽しむなら・・・正直内容自体は期待外れ。\n\nただ単に3Dを楽しむなら面白いと思います。'
 '30目にも止まらぬ早業ですな面白かったです。\n\n佐藤くんの剣心はどうなのかな～、と思っていましたが、緩急のついた演技がなかなかよかったです。\nマンガのイメージが強いので、実写で「ござる」「おろ？」とかの台詞はちょっと恥ずかしかったですが…。\n\n殺陣は見ごたえありましたね！\nマンガで描かれていた剣さばきがうまく映像化されていました。\n\n速すぎてわからないくらい。でもそれでいいのかも。\n観客に見せることを考慮してスピード緩めたら、物足りなさを感じると思います。\nDVDで見たときにスロー再生してみてもいいかも。\n\n今作はストーリーの序盤でしたが、剣心の過去に背負ったものや、人物関係などがきっちり描かれていて、納得できる内容でした。\n\nアクションシーンを楽しむつもりで見に行ったんですが、終盤の武井咲の演技に思わず涙、涙、涙。\n\n続編は未定の様ですが、ぜひ続きが見たいです。\n\n「るろうに剣心」は全巻持っていたはずなんですが、行方不明。\n中古販売したんだっけかな？\nそれすら思い出せない。\nああ、寄る年波の恐ろしさ…。\nすっかりストーリーも忘れているので、読み直したいと思います。'
 ...
 '1楽しい！映画との妙な距離感を覚えるような、間延びを感じる\nシーンが多かったです。\nけれど、そうかと思うと、急に自分のストライクな\n笑いのツボをつかれることもあって、感情がグルグル\n回って、慌しさのようなものが残りました。\nまさにドタバタ喜劇という印象･･･\n\n一緒に鑑賞していた友人の笑い声や、隣に座っていた\n年配の男性の笑い声など、周りから聞こえてくる笑い\n声と、自分の笑い声が重なる瞬間に、何度も感動を\n覚えました。嬉しくなるというか･･･\n\nこれからはしばらく、眠りに落ちる瞬間に、ずっと\n佐藤浩市さん演じる村田さんの姿が思い出されそうです･･･'
 '25月とＬ、決着編ですけど原作ファンの一人としてがっかりな最終章。\n\

In [ ]:
# Bertの事前学習済みエンコーダーを使用して5値クラス分類器を作成

class BertClassification(nn.Module):
    def __init__(self, bert, n_classes=2):
        super(BertClassification, self).__init__()

        # BERTモジュール
        self.bert = bert  # BERTモデル

        self.cls = nn.Linear(in_features=768, out_features=n_classes)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        pooled_output = self.bert(
                  input_ids, token_type_ids, attention_mask)[1]

        # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
        pooled_output = pooled_output.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        outputs = self.cls(pooled_output)

        return outputs

In [ ]:
cuda = torch.cuda.is_available()
# 学習済みモデルをダウンロード
bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertClassification(bert, n_classes=5)
# 高速化
torch.backends.cudnn.benchmark = True
if cuda:
    model.cuda()
# optimizerの設定
# optimizer = optim.Adam(model.parameters(),lr = 4e-4 ,betas=(0.9, 0.999))
optimizer = optim.AdamW(model.parameters(),lr = 1e-5 ,betas=(0.9, 0.999))
# optimizer = optim.AdamW(model.parameters(), lr = 1e-4 ,betas=(0.9, 0.999))
# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=(len(X_train) // batch_size + 1) * 1,
#     num_training_steps=(len(X_train) // batch_size + 1) * 5
# )
max_epoch = 4
total_steps = len(X_train) * max_epoch
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps = 0, # Default value in run_glue.py
    num_training_steps = total_steps)

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

In [ ]:
# # 最後以外のBertLayerモジュールのパラメータを固定(今回は速さのためにしているだけ。精度を上げるなら固定せずに実行するべき)
# for param in model.bert.encoder.layer[:-1].parameters():
#     param.requires_grad = False

In [ ]:
## 学習します。
epochs = 4
# best_train_acc = 0.0
# best_val_acc = 0.0
best_train_mae = np.inf
best_valid_mae = np.inf
for epoch in range(epochs):
    total_loss = 0.0
    accuracy = 0.0
    mae = 0.0
    model.train()

    print("epoch {} start!".format(epoch+1))
    # train
    for iter_num, (input_ids, segment_ids, attention_masks, labels) in tqdm(enumerate(train_dl), total=len(train_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, segment_ids, attention_masks, labels =\
            input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
        outputs = model(input_ids = input_ids,
                        token_type_ids = segment_ids,
                        attention_mask = attention_masks)
        loss = criterion(outputs, labels)
        pred_proba = outputs.softmax(dim=-1)
        pred = torch.argmax(pred_proba, axis=1)
        # optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        accuracy += (pred==labels).sum().item()
        mae += torch.abs(pred - labels).sum().item()
        # accuracy += (pred.indices==labels).sum().item()
        # mae += torch.abs(pred.indices - labels).sum().item()
        #50 iterごとにlossとaccuracyを表示
        if (iter_num+1) % 50 == 0:
            now_size = batch_size*(iter_num+1)
            print("{} iter loss:{:.4f} accuracy:{:.4f} MAE:{:.4f}".format(
            iter_num+1,total_loss/(iter_num+1),accuracy/now_size,mae/now_size))
            # if best_train_acc < accuracy/now_size:
            #     torch.save(model.state_dict(), './dataset/model.hdf5')
            #     best_train_acc = accuracy/now_size
            if best_train_mae > mae/now_size:
                torch.save(model.state_dict(), dataset_dir+'model_train_allin_gslww_OrdEnc_4epochs.hdf5')
                best_train_mae = mae/now_size

    total_loss /= len(train_dl)
    accuracy /= len(train_ds)
    mae /= len(train_ds)

    # valididation
    valid_total_loss = 0.0
    valid_accuracy = 0.0
    valid_mae = 0.0
    model.eval()
    for input_ids, segment_ids, attention_masks, labels in tqdm(valid_dl):
        if cuda:
            input_ids, segment_ids, attention_masks, labels =\
            input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
        with torch.no_grad():
            outputs = model(input_ids = input_ids,
                        token_type_ids = segment_ids,
                        attention_mask = attention_masks)
        pred_proba = outputs.softmax(dim=-1)
        pred = torch.argmax(pred_proba, axis=1)
        # pred = torch.max(pred_proba, dim=-1)
        loss = criterion(outputs, labels)
        valid_total_loss += loss.item()
        valid_accuracy += (pred==labels).sum().item()
        valid_mae += torch.abs(pred - labels).sum().item()
        # valid_accuracy += (pred.indices==labels).sum().item()

    valid_total_loss /= len(valid_dl)
    valid_accuracy /= len(valid_ds)
    valid_mae /= len(valid_ds)
    print("epoch:{} total loss:{:.4f}, accuracy:{:.4f}, MAE:{:.4f}, valid_total loss:{:.4f}, valid_accuracy:{:.4f}, valid_MAE:{:.4f}"\
        .format(epoch+1,total_loss,accuracy,mae,valid_total_loss,valid_accuracy,valid_mae))
    # if best_val_acc < val_accuracy:
    #     torch.save(model.state_dict(), dataset_dir+'model.hdf5')
    #     best_val_acc = val_accuracy
    if best_valid_mae > valid_mae:
        torch.save(model.state_dict(), dataset_dir+'model_valid_allin_gslww_OrdEnc_4epochs.hdf5')
        best_valid_mae = valid_mae
# torch.save(model.state_dict(), dataset_dir+'model.hdf5')

epoch 1 start!


50 iter loss:1.6753 accuracy:0.2025 MAE:1.6275
100 iter loss:1.6521 accuracy:0.1950 MAE:1.6175
150 iter loss:1.6390 accuracy:0.2017 MAE:1.5917
200 iter loss:1.6300 accuracy:0.2037 MAE:1.5850
250 iter loss:1.6190 accuracy:0.2165 MAE:1.5240
300 iter loss:1.6194 accuracy:0.2179 MAE:1.4988
350 iter loss:1.6162 accuracy:0.2243 MAE:1.4846
400 iter loss:1.6081 accuracy:0.2322 MAE:1.4622
450 iter loss:1.5964 accuracy:0.2419 MAE:1.4406
500 iter loss:1.5767 accuracy:0.2540 MAE:1.3935
550 iter loss:1.5590 accuracy:0.2668 MAE:1.3543
600 iter loss:1.5414 accuracy:0.2767 MAE:1.3210
650 iter loss:1.5232 accuracy:0.2865 MAE:1.2827
700 iter loss:1.5044 accuracy:0.2986 MAE:1.2496
750 iter loss:1.4874 accuracy:0.3105 MAE:1.2183
800 iter loss:1.4734 accuracy:0.3178 MAE:1.1944
850 iter loss:1.4587 accuracy:0.3266 MAE:1.1682
900 iter loss:1.4462 accuracy:0.3325 MAE:1.1450
950 iter loss:1.4312 accuracy:0.3407 MAE:1.1214
1000 iter loss:1.4181 accuracy:0.3483 MAE:1.0981
1050 iter loss:1.4070 accuracy:0.3525 MA


epoch:1 total loss:1.1976, accuracy:0.4614, MAE:0.7735, valid_total loss:1.0205, valid_accuracy:0.5536, valid_MAE:0.5700
epoch 2 start!


50 iter loss:0.9611 accuracy:0.5875 MAE:0.5150
100 iter loss:0.9331 accuracy:0.6025 MAE:0.4875
150 iter loss:0.9272 accuracy:0.5983 MAE:0.4858
200 iter loss:0.9381 accuracy:0.5875 MAE:0.4881
250 iter loss:0.9491 accuracy:0.5860 MAE:0.4920
300 iter loss:0.9494 accuracy:0.5883 MAE:0.4900
350 iter loss:0.9453 accuracy:0.5893 MAE:0.4925
400 iter loss:0.9472 accuracy:0.5878 MAE:0.4913
450 iter loss:0.9500 accuracy:0.5847 MAE:0.4942
500 iter loss:0.9481 accuracy:0.5843 MAE:0.4950
550 iter loss:0.9456 accuracy:0.5836 MAE:0.4943
600 iter loss:0.9464 accuracy:0.5825 MAE:0.4981
650 iter loss:0.9437 accuracy:0.5821 MAE:0.4962
700 iter loss:0.9425 accuracy:0.5807 MAE:0.4971
750 iter loss:0.9405 accuracy:0.5822 MAE:0.4962
800 iter loss:0.9403 accuracy:0.5830 MAE:0.4956
850 iter loss:0.9410 accuracy:0.5844 MAE:0.4950
900 iter loss:0.9428 accuracy:0.5843 MAE:0.4967
950 iter loss:0.9383 accuracy:0.5880 MAE:0.4921
1000 iter loss:0.9347 accuracy:0.5901 MAE:0.4895
1050 iter loss:0.9344 accuracy:0.5907 MA


epoch:2 total loss:0.9330, accuracy:0.5917, MAE:0.4829, valid_total loss:1.0246, valid_accuracy:0.5620, valid_MAE:0.5490
epoch 3 start!


50 iter loss:0.7898 accuracy:0.6600 MAE:0.3775
100 iter loss:0.7967 accuracy:0.6550 MAE:0.3775
150 iter loss:0.8079 accuracy:0.6492 MAE:0.3858
200 iter loss:0.7852 accuracy:0.6613 MAE:0.3706
250 iter loss:0.7838 accuracy:0.6595 MAE:0.3715
300 iter loss:0.7780 accuracy:0.6596 MAE:0.3721
350 iter loss:0.7742 accuracy:0.6629 MAE:0.3693
400 iter loss:0.7734 accuracy:0.6647 MAE:0.3672
450 iter loss:0.7776 accuracy:0.6625 MAE:0.3697
500 iter loss:0.7727 accuracy:0.6667 MAE:0.3663
550 iter loss:0.7761 accuracy:0.6652 MAE:0.3689
600 iter loss:0.7751 accuracy:0.6675 MAE:0.3667
650 iter loss:0.7746 accuracy:0.6671 MAE:0.3673
700 iter loss:0.7759 accuracy:0.6648 MAE:0.3707
750 iter loss:0.7731 accuracy:0.6653 MAE:0.3695
800 iter loss:0.7713 accuracy:0.6667 MAE:0.3670
850 iter loss:0.7752 accuracy:0.6672 MAE:0.3668
900 iter loss:0.7726 accuracy:0.6671 MAE:0.3669
950 iter loss:0.7723 accuracy:0.6658 MAE:0.3679
1000 iter loss:0.7704 accuracy:0.6657 MAE:0.3679
1050 iter loss:0.7705 accuracy:0.6655 MA


epoch:3 total loss:0.7816, accuracy:0.6618, MAE:0.3758, valid_total loss:1.0622, valid_accuracy:0.5464, valid_MAE:0.5386
epoch 4 start!


50 iter loss:0.5970 accuracy:0.7750 MAE:0.2375
100 iter loss:0.5944 accuracy:0.7750 MAE:0.2387
150 iter loss:0.5856 accuracy:0.7700 MAE:0.2425
200 iter loss:0.5967 accuracy:0.7681 MAE:0.2481
250 iter loss:0.6030 accuracy:0.7615 MAE:0.2565
300 iter loss:0.6059 accuracy:0.7562 MAE:0.2604
350 iter loss:0.5970 accuracy:0.7607 MAE:0.2539
400 iter loss:0.5952 accuracy:0.7597 MAE:0.2559
450 iter loss:0.6009 accuracy:0.7603 MAE:0.2558
500 iter loss:0.5963 accuracy:0.7625 MAE:0.2527
550 iter loss:0.5978 accuracy:0.7602 MAE:0.2543
600 iter loss:0.6029 accuracy:0.7588 MAE:0.2562
650 iter loss:0.6001 accuracy:0.7613 MAE:0.2533
700 iter loss:0.5977 accuracy:0.7602 MAE:0.2541
750 iter loss:0.5987 accuracy:0.7605 MAE:0.2537
800 iter loss:0.6027 accuracy:0.7586 MAE:0.2555
850 iter loss:0.6047 accuracy:0.7576 MAE:0.2568
900 iter loss:0.6075 accuracy:0.7549 MAE:0.2599
950 iter loss:0.6056 accuracy:0.7555 MAE:0.2591
1000 iter loss:0.6088 accuracy:0.7542 MAE:0.2614
1050 iter loss:0.6127 accuracy:0.7524 MA


epoch:4 total loss:0.6203, accuracy:0.7458, MAE:0.2705, valid_total loss:1.1832, valid_accuracy:0.5550, valid_MAE:0.5363


In [ ]:
model.load_state_dict(torch.load(dataset_dir+'model_valid_allin_gslww_OrdEnc_4epochs.hdf5'))

<All keys matched successfully>

In [ ]:
# test datset dataloaderを作成。
test_ds = ReviewDataset(texts=test_df["text"].values)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=1, shuffle=False)

In [ ]:
model.eval()
if cuda:
    model.cuda()
else:
    model.cpu()

# 予測を付与。
lists = []
lists_proba = []
for input_ids, segment_ids, attention_masks in tqdm(test_dl):
    if cuda:
        input_ids, segment_ids, attention_masks =\
        input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda()
    outputs = model(input_ids = input_ids,
                  token_type_ids = segment_ids,
                  attention_mask = attention_masks)
    pred_proba = outputs.softmax(dim=1)
    pred = torch.argmax(pred_proba, axis=1)
    lists += list(pred.cpu().detach().numpy()+1)
    lists_proba += list(pred_proba.cpu().detach().numpy())

In [ ]:
print(lists)
print(lists_proba)

[5, 1, 2, 2, 4, 2, 4, 2, 2, 3, 2, 1, 2, 2, 4, 2, 3, 4, 1, 3, 4, 4, 3, 3, 5, 2, 5, 5, 3, 5, 4, 1, 5, 1, 4, 2, 2, 1, 4, 3, 3, 5, 5, 4, 2, 2, 5, 1, 4, 2, 4, 4, 2, 5, 4, 3, 1, 4, 3, 1, 1, 5, 3, 2, 3, 5, 4, 2, 2, 2, 2, 2, 4, 3, 2, 5, 1, 3, 3, 5, 1, 2, 1, 3, 4, 3, 3, 3, 4, 3, 2, 4, 1, 2, 2, 2, 4, 2, 2, 2, 5, 4, 4, 3, 5, 2, 3, 4, 5, 4, 5, 2, 4, 5, 5, 2, 5, 3, 3, 5, 3, 4, 4, 3, 4, 4, 3, 2, 4, 5, 4, 1, 5, 5, 3, 1, 3, 2, 5, 1, 3, 1, 2, 1, 3, 5, 3, 1, 5, 1, 3, 1, 4, 5, 5, 3, 5, 4, 3, 1, 2, 1, 2, 3, 1, 2, 5, 1, 3, 1, 5, 4, 1, 2, 2, 2, 5, 3, 2, 2, 3, 4, 1, 1, 4, 3, 2, 4, 2, 1, 2, 3, 4, 1, 4, 2, 4, 2, 1, 1, 5, 2, 4, 1, 3, 4, 2, 4, 5, 3, 1, 2, 3, 3, 3, 3, 1, 2, 3, 3, 5, 1, 2, 5, 3, 3, 5, 4, 2, 1, 3, 1, 3, 1, 1, 1, 3, 5, 4, 4, 5, 2, 4, 2, 4, 4, 3, 4, 2, 1, 5, 2, 5, 5, 5, 3, 3, 3, 1, 4, 5, 2, 2, 2, 2, 4, 4, 1, 2, 4, 1, 5, 2, 1, 1, 5, 1, 4, 3, 2, 4, 3, 3, 2, 3, 1, 2, 3, 5, 5, 1, 1, 1, 5, 1, 2, 4, 5, 2, 5, 2, 2, 2, 5, 5, 4, 3, 4, 1, 4, 2, 5, 4, 1, 4, 4, 4, 3, 5, 5, 1, 3, 5, 3, 2, 3, 4, 3, 4, 3, 1, 3, 2, 

In [ ]:
# test_df["Label"]=lists
# submission_df=test_df[["Id","Label"]]
# submission_df.to_csv("sample_submission.csv",index=False)

In [ ]:
result = "\n".join([str(rating) for rating in lists])
open(dataset_dir+"test_pred_bert_allin_gslww_OrdEnc_4epochs.txt", "w").write(result)
pd.DataFrame(lists_proba).to_csv(dataset_dir+"bert_pred_proba_allin_gslww_OrdEnc_4epochs.csv")

In [ ]:
## 学習します。
epochs = 4
plusAlpha = 4
# best_train_acc = 0.0
# best_val_acc = 0.0
best_train_mae = np.inf
best_valid_mae = np.inf
for epoch in range(epochs, epochs+plusAlpha):
    total_loss = 0.0
    accuracy = 0.0
    mae = 0.0
    model.train()

    print("epoch {} start!".format(epoch+1))
    # train
    for iter_num, (input_ids, segment_ids, attention_masks, labels) in tqdm(enumerate(train_dl), total=len(train_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, segment_ids, attention_masks, labels =\
            input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
        outputs = model(input_ids = input_ids,
                        token_type_ids = segment_ids,
                        attention_mask = attention_masks)
        loss = criterion(outputs, labels)
        pred_proba = outputs.softmax(dim=-1)
        pred = torch.argmax(pred_proba, axis=1)
        # optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        accuracy += (pred==labels).sum().item()
        mae += torch.abs(pred - labels).sum().item()
        # accuracy += (pred.indices==labels).sum().item()
        # mae += torch.abs(pred.indices - labels).sum().item()
        #50 iterごとにlossとaccuracyを表示
        if (iter_num+1) % 50 == 0:
            now_size = batch_size*(iter_num+1)
            print("{} iter loss:{:.4f} accuracy:{:.4f} MAE:{:.4f}".format(
            iter_num+1,total_loss/(iter_num+1),accuracy/now_size,mae/now_size))
            # if best_train_acc < accuracy/now_size:
            #     torch.save(model.state_dict(), './dataset/model.hdf5')
            #     best_train_acc = accuracy/now_size
            if best_train_mae > mae/now_size:
                torch.save(model.state_dict(), dataset_dir+'model_train_allin_gslww_OrdEnc_4epochsPlusAlpha.hdf5')
                best_train_mae = mae/now_size

    total_loss /= len(train_dl)
    accuracy /= len(train_ds)
    mae /= len(train_ds)

    # valididation
    valid_total_loss = 0.0
    valid_accuracy = 0.0
    valid_mae = 0.0
    model.eval()
    for input_ids, segment_ids, attention_masks, labels in tqdm(valid_dl):
        if cuda:
            input_ids, segment_ids, attention_masks, labels =\
            input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
        with torch.no_grad():
            outputs = model(input_ids = input_ids,
                        token_type_ids = segment_ids,
                        attention_mask = attention_masks)
        pred_proba = outputs.softmax(dim=-1)
        pred = torch.argmax(pred_proba, axis=1)
        # pred = torch.max(pred_proba, dim=-1)
        loss = criterion(outputs, labels)
        valid_total_loss += loss.item()
        valid_accuracy += (pred==labels).sum().item()
        valid_mae += torch.abs(pred - labels).sum().item()
        # valid_accuracy += (pred.indices==labels).sum().item()

    valid_total_loss /= len(valid_dl)
    valid_accuracy /= len(valid_ds)
    valid_mae /= len(valid_ds)
    print("epoch:{} total loss:{:.4f}, accuracy:{:.4f}, MAE:{:.4f}, valid_total loss:{:.4f}, valid_accuracy:{:.4f}, valid_MAE:{:.4f}"\
        .format(epoch+1,total_loss,accuracy,mae,valid_total_loss,valid_accuracy,valid_mae))
    # if best_val_acc < val_accuracy:
    #     torch.save(model.state_dict(), dataset_dir+'model.hdf5')
    #     best_val_acc = val_accuracy
    if best_valid_mae > valid_mae:
        torch.save(model.state_dict(), dataset_dir+'model_valid_allin_gslww_OrdEnc_4epochsPlusAlpha.hdf5')
        best_valid_mae = valid_mae
# torch.save(model.state_dict(), dataset_dir+'model.hdf5')

epoch 5 start!


50 iter loss:0.4970 accuracy:0.8125 MAE:0.2025
100 iter loss:0.4990 accuracy:0.8125 MAE:0.2000
150 iter loss:0.5064 accuracy:0.8050 MAE:0.2092
200 iter loss:0.4820 accuracy:0.8137 MAE:0.1981
250 iter loss:0.4671 accuracy:0.8175 MAE:0.1935
300 iter loss:0.4638 accuracy:0.8192 MAE:0.1908
350 iter loss:0.4613 accuracy:0.8207 MAE:0.1879
400 iter loss:0.4553 accuracy:0.8231 MAE:0.1862
450 iter loss:0.4549 accuracy:0.8244 MAE:0.1842
500 iter loss:0.4561 accuracy:0.8245 MAE:0.1835
550 iter loss:0.4546 accuracy:0.8245 MAE:0.1834
600 iter loss:0.4551 accuracy:0.8229 MAE:0.1854
650 iter loss:0.4547 accuracy:0.8235 MAE:0.1846
700 iter loss:0.4576 accuracy:0.8209 MAE:0.1871
750 iter loss:0.4578 accuracy:0.8222 MAE:0.1855
800 iter loss:0.4584 accuracy:0.8214 MAE:0.1858
850 iter loss:0.4584 accuracy:0.8207 MAE:0.1871
900 iter loss:0.4608 accuracy:0.8196 MAE:0.1885
950 iter loss:0.4615 accuracy:0.8192 MAE:0.1895
1000 iter loss:0.4641 accuracy:0.8185 MAE:0.1905
1050 iter loss:0.4662 accuracy:0.8174 MA


epoch:5 total loss:0.4800, accuracy:0.8096, MAE:0.1991, valid_total loss:1.3196, valid_accuracy:0.5494, valid_MAE:0.5421
epoch 6 start!


50 iter loss:0.3308 accuracy:0.8750 MAE:0.1325
100 iter loss:0.3260 accuracy:0.8812 MAE:0.1225
150 iter loss:0.3360 accuracy:0.8725 MAE:0.1308
200 iter loss:0.3347 accuracy:0.8750 MAE:0.1294
250 iter loss:0.3385 accuracy:0.8730 MAE:0.1310
300 iter loss:0.3333 accuracy:0.8754 MAE:0.1288
350 iter loss:0.3282 accuracy:0.8789 MAE:0.1268
400 iter loss:0.3349 accuracy:0.8772 MAE:0.1288
450 iter loss:0.3390 accuracy:0.8747 MAE:0.1314
500 iter loss:0.3353 accuracy:0.8742 MAE:0.1315
550 iter loss:0.3354 accuracy:0.8750 MAE:0.1302
600 iter loss:0.3406 accuracy:0.8735 MAE:0.1313
650 iter loss:0.3402 accuracy:0.8729 MAE:0.1321
700 iter loss:0.3406 accuracy:0.8721 MAE:0.1329
750 iter loss:0.3431 accuracy:0.8717 MAE:0.1335
800 iter loss:0.3414 accuracy:0.8714 MAE:0.1339
850 iter loss:0.3402 accuracy:0.8721 MAE:0.1332
900 iter loss:0.3408 accuracy:0.8725 MAE:0.1325
950 iter loss:0.3457 accuracy:0.8708 MAE:0.1342
1000 iter loss:0.3467 accuracy:0.8708 MAE:0.1344
1050 iter loss:0.3486 accuracy:0.8696 MA


epoch:6 total loss:0.3576, accuracy:0.8630, MAE:0.1418, valid_total loss:1.4794, valid_accuracy:0.5550, valid_MAE:0.5430
epoch 7 start!


50 iter loss:0.2333 accuracy:0.9275 MAE:0.0800
100 iter loss:0.2572 accuracy:0.9137 MAE:0.0938
150 iter loss:0.2485 accuracy:0.9133 MAE:0.0925
200 iter loss:0.2636 accuracy:0.9069 MAE:0.1013
250 iter loss:0.2643 accuracy:0.9050 MAE:0.1030
300 iter loss:0.2589 accuracy:0.9083 MAE:0.0983
350 iter loss:0.2518 accuracy:0.9096 MAE:0.0964
400 iter loss:0.2504 accuracy:0.9100 MAE:0.0953
450 iter loss:0.2507 accuracy:0.9097 MAE:0.0961
500 iter loss:0.2557 accuracy:0.9077 MAE:0.0978
550 iter loss:0.2612 accuracy:0.9064 MAE:0.0993
600 iter loss:0.2615 accuracy:0.9062 MAE:0.0990
650 iter loss:0.2595 accuracy:0.9069 MAE:0.0979
700 iter loss:0.2602 accuracy:0.9062 MAE:0.0986
750 iter loss:0.2582 accuracy:0.9060 MAE:0.0985
800 iter loss:0.2568 accuracy:0.9053 MAE:0.0991
850 iter loss:0.2584 accuracy:0.9037 MAE:0.1004
900 iter loss:0.2614 accuracy:0.9015 MAE:0.1025
950 iter loss:0.2633 accuracy:0.9014 MAE:0.1028
1000 iter loss:0.2660 accuracy:0.9009 MAE:0.1036
1050 iter loss:0.2669 accuracy:0.9005 MA


epoch:7 total loss:0.2679, accuracy:0.8973, MAE:0.1065, valid_total loss:1.7253, valid_accuracy:0.5443, valid_MAE:0.5411
epoch 8 start!


50 iter loss:0.2116 accuracy:0.9150 MAE:0.0850
100 iter loss:0.2003 accuracy:0.9275 MAE:0.0737
150 iter loss:0.1954 accuracy:0.9292 MAE:0.0717
200 iter loss:0.2087 accuracy:0.9219 MAE:0.0788
250 iter loss:0.2149 accuracy:0.9200 MAE:0.0810
300 iter loss:0.2060 accuracy:0.9242 MAE:0.0767
350 iter loss:0.1980 accuracy:0.9271 MAE:0.0736
400 iter loss:0.1971 accuracy:0.9269 MAE:0.0737
450 iter loss:0.2014 accuracy:0.9250 MAE:0.0764
500 iter loss:0.1985 accuracy:0.9263 MAE:0.0750
550 iter loss:0.1951 accuracy:0.9275 MAE:0.0736
600 iter loss:0.1917 accuracy:0.9292 MAE:0.0719
650 iter loss:0.1902 accuracy:0.9296 MAE:0.0719
700 iter loss:0.1900 accuracy:0.9302 MAE:0.0718
750 iter loss:0.1884 accuracy:0.9315 MAE:0.0703
800 iter loss:0.1890 accuracy:0.9309 MAE:0.0709
850 iter loss:0.1912 accuracy:0.9293 MAE:0.0725
900 iter loss:0.1924 accuracy:0.9294 MAE:0.0724
950 iter loss:0.1921 accuracy:0.9295 MAE:0.0722
1000 iter loss:0.1910 accuracy:0.9296 MAE:0.0720
1050 iter loss:0.1900 accuracy:0.9305 MA


epoch:8 total loss:0.1958, accuracy:0.9270, MAE:0.0753, valid_total loss:1.8767, valid_accuracy:0.5553, valid_MAE:0.5363


In [ ]:
model.load_state_dict(torch.load(dataset_dir+'model_valid_allin_gslww_OrdEnc_4epochsPlusAlpha.hdf5'))

<All keys matched successfully>

In [ ]:
# test datset dataloaderを作成。
test_ds = ReviewDataset(texts=test_df["text"].values)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=1, shuffle=False)

In [ ]:
model.eval()
if cuda:
    model.cuda()
else:
    model.cpu()

# 予測を付与。
lists = []
lists_proba = []
for input_ids, segment_ids, attention_masks in tqdm(test_dl):
    if cuda:
        input_ids, segment_ids, attention_masks =\
        input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda()
    outputs = model(input_ids = input_ids,
                  token_type_ids = segment_ids,
                  attention_mask = attention_masks)
    pred_proba = outputs.softmax(dim=1)
    pred = torch.argmax(pred_proba, axis=1)
    lists += list(pred.cpu().detach().numpy()+1)
    lists_proba += list(pred_proba.cpu().detach().numpy())

In [ ]:
result = "\n".join([str(rating) for rating in lists])
open(dataset_dir+"test_pred_bert_allin_gslww_OrdEnc_4epochsPlusAlpha.txt", "w").write(result)
pd.DataFrame(lists_proba).to_csv(dataset_dir+"bert_pred_proba_allin_gslww_OrdEnc_4epochsPlusAlpha.csv")